In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
df_train = pd.read_csv(r"drive/My Drive/bitrate_model_final/bitrate_training.csv")
df_test = pd.read_csv(r"drive/My Drive/bitrate_model_final/bitrate_testing.csv")
df_validation = pd.read_csv(r"drive/My Drive/bitrate_model_final/bitrate_validation.csv")

In [3]:
#pd.set_option('display.max_rows', 1000)
df_train.isna().sum()

Unnamed: 0                   0
filename                     0
size-bytes                   0
height                       0
width                        0
framespersecond              0
duration                     0
i_frame                      0
b_frame                      0
p_frame                      0
text                         0
Bitrate                      0
bit_rate_mode                0
channel_layout               0
channel_positions            0
channel_s                    0
frame_count                  0
proportion_of_this_stream    0
stream_size                  0
other_bit_rate_mode          0
dtype: int64

In [4]:
#df_validation
df_train = df_train.iloc[: , 1:]
df_test = df_test.iloc[: , 1:]
df_validation = df_validation.iloc[: , 1:]


In [5]:
df_train["text"] = df_train["text"].str.lower()
df_test["text"] = df_test["text"].str.lower()
df_validation["text"] = df_validation["text"].str.lower()

In [6]:
df_train["len_text"] = df_train['text'].str.len() #df_test
df_test["len_text"] = df_test['text'].str.len()
df_validation["len_text"] = df_validation['text'].str.len()


In [7]:
df_train.drop(columns=['filename', 'text'], axis=1, inplace=True)
df_test.drop(columns=['filename', 'text'], axis=1, inplace=True)
df_validation.drop(columns=['filename', 'text'], axis=1, inplace=True)


In [8]:
#df_train.isna().sum()
df_train = df_train.reset_index()
df_test = df_test.reset_index()
df_validation = df_validation.reset_index()

In [9]:
#df_train["label"].replace({"CBR": 1, "VBR": 0}, inplace=True)
encode_train = pd.get_dummies(df_train, columns = ['bit_rate_mode', 'channel_layout','channel_positions','other_bit_rate_mode'])
encode_test = pd.get_dummies(df_test, columns = ['bit_rate_mode', 'channel_layout','channel_positions','other_bit_rate_mode'])
encode_val = pd.get_dummies(df_validation, columns = ['bit_rate_mode', 'channel_layout','channel_positions','other_bit_rate_mode'])


In [10]:
#channel_layout_C L R Ls Rs LFE  channel_positions_Front:L C R, Side: L R, LFE

encode_train.drop(columns=['channel_layout_C L R Ls Rs LFE', 'channel_positions_Front: L C R, Side: L R, LFE'], axis=1, inplace=True)
#encode_test.drop(columns=['channel_layout_C L R Ls Rs LFE', 'channel_positions_Front: L C R, Side: L R, LFE'], axis=1, inplace=True)
encode_val.drop(columns=['channel_layout_C L R Ls Rs LFE', 'channel_positions_Front: L C R, Side: L R, LFE'], axis=1, inplace=True)


In [11]:
 dff=encode_train.append(encode_test)

In [12]:
dfff=dff.append(encode_val)

In [13]:
dfff

,index,size-bytes,height,width,framespersecond,duration,i_frame,b_frame,p_frame,Bitrate,channel_s,frame_count,proportion_of_this_stream,stream_size,len_text,bit_rate_mode_CBR,bit_rate_mode_VBR,channel_layout_C,channel_layout_L R,channel_positions_Front: C,channel_positions_Front: L R,other_bit_rate_mode_Constant,other_bit_rate_mode_Variable
0,0,125469259.0,144.0,256.0,25,5202.640000,2255,64571,63240,192,2,224065,0.66347,83244466,6395,1,0,0,1,0,1,1,0
1,1,156812882.0,240.0,426.0,23,6375.608696,2536,70413,73690,205,2,263400,0.62404,97858176,12299,1,0,0,1,0,1,1,0
2,2,518875095.0,546.0,1280.0,24,4836.000000,2185,63281,50598,858,2,208270,0.14912,77376319,11292,0,1,0,1,0,1,0,1
3,3,520279708.0,546.0,1280.0,24,4836.000000,2185,63281,50598,860,2,208273,0.14872,77377433,11289,1,0,0,1,0,1,1,0
4,4,282485177.0,342.0,640.0,24,5006.916667,1864,79409,38893,451,2,215632,0.21270,60083583,11336,0,1,0,1,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193,193,80518345.0,720.0,1280.0,23,618.130435,236,8242,5739,1086,2,25537,0.11783,9487487,980,0,1,0,1,0,1,0,1
194,194,243886613.0,1080.0,1920.0,23,636.956522,300,7364,6986,3192,2,26317,0.04009,9777272,697,1,0,0,1,0,1,1,0
195,195,62480217.0,720.0,1280.0,29,302.896552,110,3324,5350,1705,2,12623,0.07365,4601535,114,0,1,0,1,0,1,0,1
196,196,21342015.0,668.0,1280.0,23,134.391304,34,1915,1142,1324,2,5553,0.09485,2024311,359,0,1,0,1,0,1,0,1


In [14]:
dfff = dfff.iloc[: , 1:]


In [15]:
dfff.columns

Index(['size-bytes', 'height', 'width', 'framespersecond', 'duration',
       'i_frame', 'b_frame', 'p_frame', 'Bitrate', 'channel_s', 'frame_count',
       'proportion_of_this_stream', 'stream_size', 'len_text',
       'bit_rate_mode_CBR', 'bit_rate_mode_VBR', 'channel_layout_C',
       'channel_layout_L R', 'channel_positions_Front: C',
       'channel_positions_Front: L R', 'other_bit_rate_mode_Constant',
       'other_bit_rate_mode_Variable'],
      dtype='object')

In [16]:
dfff.rename(columns={'size-bytes': 'size_bytes','Bitrate':'bitrate' ,'channel_layout_L R': 'channel_layout_l_r','bit_rate_mode_CBR':'bit_rate_mode_cbr', 'bit_rate_mode_VBR':'bit_rate_mode_vbr', 'channel_layout_C':'channel_layout_c',
'channel_layout_L R':'channel_layout_l_r','other_bit_rate_mode_Constant':'other_bit_rate_mode_constant','other_bit_rate_mode_Variable':'other_bit_rate_mode_variable','channel_positions_Front: C':'channel_positions_front_c',
       'channel_positions_Front: L R':'channel_positions_front_l_r'}, inplace=True)


In [17]:
dfff.columns

Index(['size_bytes', 'height', 'width', 'framespersecond', 'duration',
       'i_frame', 'b_frame', 'p_frame', 'bitrate', 'channel_s', 'frame_count',
       'proportion_of_this_stream', 'stream_size', 'len_text',
       'bit_rate_mode_cbr', 'bit_rate_mode_vbr', 'channel_layout_c',
       'channel_layout_l_r', 'channel_positions_front_c',
       'channel_positions_front_l_r', 'other_bit_rate_mode_constant',
       'other_bit_rate_mode_variable'],
      dtype='object')

In [18]:
dfff1 = dfff[['size_bytes', 'height', 'width', 'framespersecond', 'duration',
       'i_frame', 'b_frame', 'p_frame', 'channel_s', 'frame_count',
       'proportion_of_this_stream', 'stream_size', 'len_text',
       'bit_rate_mode_cbr', 'bit_rate_mode_vbr', 'channel_layout_c',
       'channel_layout_l_r', 'channel_positions_front_c',
       'channel_positions_front_l_r', 'other_bit_rate_mode_constant',
       'other_bit_rate_mode_variable','bitrate']]


In [21]:
dfff1.to_csv("drive/My Drive/df_bitrate.csv", encoding='utf-8')

In [22]:

labels=dfff1['bitrate'].values 
#dfff.drop('Bitrate',axis=1,inplace=True)
#df.drop('column_name', axis=1, inplace=True)
features = dfff1.iloc[:,0:21].values 
import numpy as np
#features = dfff
labelss=np.ravel(labels)

In [23]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler().fit(features)

features = scaler.transform(features)

In [24]:
import pandas as pd
import numpy as np
np.random.seed(333)
import joblib
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.preprocessing import StandardScaler

from tensorflow.keras.models import Sequential
from tensorflow.keras import regularizers
from tensorflow import keras
from tensorflow.keras.layers import (
    Conv1D,
    Dense,
    Dropout,
    Flatten,
    MaxPooling1D
)

from tensorflow.keras.metrics import RootMeanSquaredError 
from tensorflow.keras import optimizers
import matplotlib.pyplot as plt


import matplotlib.pyplot as plt

In [25]:
dfff1.isna().sum()

size_bytes                      0
height                          0
width                           0
framespersecond                 0
duration                        0
i_frame                         0
b_frame                         0
p_frame                         0
channel_s                       0
frame_count                     0
proportion_of_this_stream       0
stream_size                     0
len_text                        0
bit_rate_mode_cbr               0
bit_rate_mode_vbr               0
channel_layout_c                0
channel_layout_l_r              0
channel_positions_front_c       0
channel_positions_front_l_r     0
other_bit_rate_mode_constant    0
other_bit_rate_mode_variable    0
bitrate                         0
dtype: int64

In [26]:
labelss

array([ 192,  205,  858, ..., 1705, 1324, 1105])

In [27]:
features.shape

(1383, 21)

In [28]:
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.losses import sparse_categorical_crossentropy
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import KFold
import numpy as np
from tensorflow.keras.metrics import RootMeanSquaredError 
from tensorflow.keras import optimizers
import matplotlib.pyplot as plt
from tensorflow.keras import optimizers
import tensorflow.compat.v1 as tf
# Model configuration



verbosity = 1
num_folds = 10

inputs = features
targets = labelss

# Determine shape of the data
input_shape=[(features.shape)[1]]



# Define per-fold score containers
acc_per_fold = []
loss_per_fold = []


# Define the K-fold Cross Validator
kfold = KFold(n_splits=num_folds, shuffle=True)

# K-fold Cross Validation model evaluation
fold_no = 1
for train, test in kfold.split(inputs, targets):

  # Define the model architecture
  model = Sequential([
    Dense(21, activation='relu',input_shape=input_shape),
    #Dense(64,activation='relu',kernel_regularizer=keras.regularizers.l2(l=0.001)),
    Dense(64,activation='relu'),
    Dense(128,activation='relu'),
    Dense(256,activation='relu'),
    Dense(512,activation='relu'),
    Dense(512,activation='relu'),
    Dense(1024,activation='relu'),
    Dense(256,activation='relu'),
    Dense(128,activation='relu'),
    Dense(64,activation='relu'),
    Dense(64,activation='relu'),
    Dense(32,activation='relu'),
    Dense(8,activation='relu'),
    Dense(1,activation='linear')

  ])

  # Compile the model
  model.compile(loss='mean_squared_error', optimizer=optimizers.Adam(lr=0.001), metrics=[tf.keras.metrics.RootMeanSquaredError(name='rmse')])

  # Generate a print
  print('------------------------------------------------------------------------')
  print(f'Training for fold {fold_no} ...')
 

  history=model.fit(inputs[train], targets[train], epochs=350, batch_size=500, verbose=1)

  # Fit data to model

  # Generate generalization metrics
  scores = model.evaluate(inputs[test], targets[test], verbose=0)
  print(f'Score for fold {fold_no}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]}')
  acc_per_fold.append(scores[1])
  loss_per_fold.append(scores[0])

  # Increase fold number
  fold_no = fold_no + 1

# == Provide average scores ==
print('------------------------------------------------------------------------')
print('Score per fold')
for i in range(0, len(acc_per_fold)):
  print('------------------------------------------------------------------------')
  print(f'> Fold {i+1} - Loss: {loss_per_fold[i]} - RMSE: {acc_per_fold[i]}')
print('------------------------------------------------------------------------')
print('Average scores for all folds:')
print(f'> RMSE: {np.mean(acc_per_fold)} (+- {np.std(acc_per_fold)})')
print(f'> Loss: {np.mean(loss_per_fold)}')
print('------------------------------------------------------------------------')

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Streaming output truncated to the last 5000 lines.
3/3 [==============================] - 0s 36ms/step - loss: 784.9271 - rmse: 28.0166
Epoch 326/350
3/3 [==============================] - 0s 38ms/step - loss: 707.9512 - rmse: 26.6074
Epoch 327/350
3/3 [==============================] - 0s 38ms/step - loss: 967.2508 - rmse: 31.1007
Epoch 328/350
3/3 [==============================] - 0s 36ms/step - loss: 1082.3164 - rmse: 32.8986
Epoch 329/350
3/3 [==============================] - 0s 41ms/step - loss: 609.9305 - rmse: 24.6968
Epoch 330/350
3/3 [==============================] - 0s 39ms/step - loss: 528.9504 - rmse: 22.9989
Epoch 331/350
3/3 [==============================] - 0s 38ms/step - loss: 433.6974 - rmse: 20.8254
Epoch 332/350
3/3 [==============================] - 0s 37ms/step - loss: 392.5277 - rmse: 19.8123
Epoch 333/350
3/3 [==============================] - 0s 38ms/step - loss: 388.0092 - rmse: 19.6980
Epoch 334/350
3/3 [==============================] - 0s 37ms/step - los

In [29]:
model.summary()

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_126 (Dense)           (None, 21)                462       
                                                                 
 dense_127 (Dense)           (None, 64)                1408      
                                                                 
 dense_128 (Dense)           (None, 128)               8320      
                                                                 
 dense_129 (Dense)           (None, 256)               33024     
                                                                 
 dense_130 (Dense)           (None, 512)               131584    
                                                                 
 dense_131 (Dense)           (None, 512)               262656    
                                                                 
 dense_132 (Dense)           (None, 1024)             